In [ ]:
# https://data.mendeley.com/datasets/rz3xg97rm5/1 (dataset source)
#https://www.kaggle.com/datasets/rtatman/urdu-stopwords-list?select=stopwords-ur.txt(stopwords source)
#https://sourceforge.net/projects/resource-for-urdu-stemmer/files/Urdu%20Affix%20lists.pdf/download (stop words,suffix,prefifix)

# Import Libraries/Packages

In [57]:
import pandas as pd
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from nltk.util import ngrams
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np


In [58]:
df=pd.read_csv('C:/Users/admin/OneDrive/Desktop/SEMESTER7/NLP_ass#1/Urdu Tweets Dataset.csv',encoding='utf-8-sig')

df = df.drop(['Category','Id','Emoticon'], axis=1)  #not needed


# 1) STOP WORDS REMOVAL

In [59]:
urdu_stopwords = [
    "آئی", "آئے", "آج", "آخر", "آخرکبر", "آدهی", "آًب", "آٹھ", "آیب", "اة", 
    "اخبزت", "اختتبم", "ادھر", "ارد", "اردگرد", "ارکبى", "اش", "اضتعوبل", 
    "اضتعوبلات", "اضطرذ", "اضکب", "اضکی", "اضکے", "اطراف", "اغیب", "افراد", 
    "الگ", "اور", "اوًچب", "اوًچبئی", "اوًچی", "اوًچے", "اى", "اً", "اًذر", 
    "اًہیں", "اٹھبًب", "اپٌب", "اپٌے", "اچھب", "اچھی", "اچھے", "اکثر", "اکٹھب", 
    "اکٹھی", "اکٹھے", "اکیلا", "اکیلی", "اکیلے", "اگرچہ", "اہن", "ایطے", 
    "ایک", "ب", "ت", "تبزٍ", "تت", "تر", "ترتیت", "تریي", "تعذاد", "تن", "تو", 
    "توبم", "توہی", "توہیں", "تٌہب", "تک", "تھب", "تھوڑا", "تھوڑی", "تھوڑے", 
    "تھی", "تھے", "تیي", "ثب", "ثبئیں", "ثبترتیت", "ثبری", "ثبرے", "ثبعث", 
    "ثبلا", "ثبلترتیت", "ثبہر", "ثدبئے", "ثرآں", "ثراں", "ثرش", "ثعذ", "ثغیر", 
    "ثلٌذ", "ثلٌذوثبلا", "ثلکہ", "ثي", "ثٌب", "ثٌبرہب", "ثٌبرہی", "ثٌبرہے", 
    "ثٌبًب", "ثٌذ", "ثٌذکرو", "ثٌذکرًب", "ثٌذی", "ثڑا", "ثڑوں", "ثڑی", "ثڑے", 
    "ثھر", "ثھرا", "ثھراہوا", "ثھرپور", "ثھی", "ثہت", "ثہتر", "ثہتری", 
    "ثہتریي", "ثیچ", "ج", "خب", "خبرہب", "خبرہی", "خبرہے", "خبهوظ", "خبًب", 
    "خبًتب", "خبًتی", "خبًتے", "خبًٌب", "خت", "ختن", "خجکہ", "خص", "خططرذ", 
    "خلذی", "خو", "خواى", "خوًہی", "خوکہ", "خٌبة", "خگہ", "خگہوں", "خگہیں", 
    "خیطب", "خیطبکہ", "در", "درخبت", "درخہ", "درخے", "درزقیقت", "درضت", 
    "دش", "دفعہ", "دلچطپ", "دلچطپی", "دلچطپیبں", "دو", "دور", "دوراى", 
    "دوضرا", "دوضروں", "دوضری", "دوضرے", "دوًوں", "دکھبئیں", "دکھبتب", 
    "دکھبتی", "دکھبتے", "دکھبو", "دکھبًب", "دکھبیب", "دی", "دیب", "دیتب", 
    "دیتی", "دیتے", "دیر", "دیٌب", "دیکھو", "دیکھٌب", "دیکھی", "دیکھیں", 
    "دے", "ر", "راضتوں", "راضتہ", "راضتے", "رریعہ", "رریعے", "رکي", "رکھ", 
    "رکھب", "رکھتب", "رکھتبہوں", "رکھتی", "رکھتے", "رکھی", "رکھے", "رہب", 
    "رہی", "رہے", "ز", "زبصل", "زبضر", "زبل", "زبلات", "زبلیہ", "زصوں", 
    "زصہ", "زصے", "زقبئق", "زقیتیں", "زقیقت", "زکن", "زکویہ", "زیبدٍ", 
    "صبف", "صسیر", "صفر", "صورت", "صورتسبل", "صورتوں", "صورتیں", "ض", "ضبت", 
    "ضبتھ", "ضبدٍ", "ضبرا", "ضبرے", "ضبل", "ضبلوں", "ضت", "ضرور", "ضرورت", 
    "ضروری", "ضلطلہ", "ضوچ", "ضوچب", "ضوچتب", "ضوچتی", "ضوچتے", "ضوچو", 
    "ضوچٌب", "ضوچی", "ضوچیں", "ضکب", "ضکتب", "ضکتی", "ضکتے", "ضکٌب", 
    "ضکی", "ضکے", "ضیذھب", "ضیذھی", "ضیذھے", "ضیکٌڈ", "ضے", "طرف", "طریق", 
    "طریقوں", "طریقہ", "طریقے", "طور", "طورپر", "ظبہر", "ع", "عذد", "عظین", 
    "علاقوں", "علاقہ", "علاقے", "علاوٍ", "عووهی", "غبیذ", "غخص", "غذ", 
    "غروع", "غروعبت", "غے", "فرد", "فی", "ق", "قجل", "قجیلہ", "قطن", "لئے", 
    "لا", "لازهی", "لو", "لوجب", "لوجی", "لوجے", "لوسبت", "لوسہ", "لوگ", 
    "لوگوں", "لڑکپي", "لگتب", "لگتی", "لگتے", "لگٌب", "لگی", "لگیں", 
    "لگے", "لی", "لیب", "لیٌب", "لیں", "لے", "هتعلق", "هختلف", 
    "هسترم", "هسترهہ", "هسطوش", "هسیذ", "هطئلہ", "هطئلے", "هطبئل", 
    "هطتعول", "هطلق", "هعلوم", "هػتول", "هلا", "هوکي", "هوکٌبت", "هوکٌہ", 
    "هٌبضت", "هڑا", "هڑًب", "هڑے", "هکول", "هگر", "هہرثبى", "هیرا", "هیری", 
    "هیرے", "هیں", "و", "وار", "والے", "وٍ", "ًئی", "ًئے", "ًب", "ًبپطٌذ", 
    "ًبگسیر", "ًطجت", "ًقطہ", "ًو", "ًوخواى", "ًکبلٌب", "ًکتہ", "ًہ", 
    "ًہیں", "ًیب", "ًے", "ٓ آش", "ٹھیک", "پبئے", "پبش", "پبًب", "پبًچ", "پر", 
    "پراًب", "پطٌذ", "پل", "پورا", "پوچھب", "پوچھتب", "پوچھتی", "پوچھتے", 
    "پوچھو", "پوچھوں", "پوچھٌب", "پوچھیں", "پچھلا", "پھر", "پہلا", "پہلی", 
    "پہلےضی", "پہلےضے", "پہلےضےہی", "پیع", "چبر", "چبہب", "چبہٌب", 
    "چبہے", "چلا", "چلو", "چلیں", "چلے", "چکب", "چکی", "چکیں", "چکے", 
    "چھوٹب", "چھوٹوں", "چھوٹی", "چھوٹے", "چھہ", "چیسیں", "ڈھوًڈا", 
    "ڈھوًڈلیب", "ڈھوًڈو", "ڈھوًڈًب", "ڈھوًڈی", "ڈھوًڈیں", "ک", "کئی", "کئے", 
    "کب", "کبفی", "کبم", "کت", "کجھی", "کرا", "کرتب", "کرتبہوں", "کرتی", 
    "کرتے", "کرتےہو", "کررہب", "کررہی", "کررہے", "کرو", "کرًب", "کریں", 
    "کرے", "کطی", "کل", "کن", "کوئی", "کوتر", "کورا", "کوروں", "کورٍ", 
    "کورے", "کوطي", "کوى", "کوًطب", "کوًطی", "کوًطے", "کھولا", "کھولو", 
    "کھولٌب", "کھولی", "کھولیں", "کھولے", "کہ", "کہب", "کہتب", "کہتی", 
    "کہتے", "کہو", "کہوں", "کہٌب", "کہی", "کہیں", "کہے", "کی", "کیب", 
    "کیطب", "کیطرف", "کیطے", "کیلئے", "کیوًکہ", "کیوں", "کیے", "کے", 
    "کےثعذ", "کےرریعے", "گئی", "گئے", "گب", "گرد", "گروٍ", "گروپ", 
    "گروہوں", "گٌتی", "گی", "گیب", "گے", "ہر", "ہن", "ہو", "ہوئی", "ہوئے", 
    "ہوا", "ہوبرا", "ہوبری", "ہوبرے", "ہوتب", "ہوتی", "ہوتے", "ہورہب", 
    "ہورہی", "ہورہے", "ہوضکتب", "ہوضکتی", "ہوضکتے", "ہوًب", "ہوًی", 
    "ہوًے", "ہوچکب", "ہوچکی", "ہوچکے", "ہوگئی", "ہوگئے", "ہوگیب", "ہوں", 
    "ہی", "ہیں", "ہے", "ی", "یقیٌی", "یہ", "یہبں",
]
urdu_stopwords1 = [
     'آئی', 'آئے', 'آج', 'آدمی', 'آپ', 'اپنا', 'اپنی', 'اپنے', 'اکثر', 'اگر', 'اور', 'ایسا', 'ایسی',
     'ایسے', 'ایک', 'بار', 'بعض', 'بعد', 'بتایا', 'بھی', 'بنا', 'بنانا', 'بنایا', 'بنے', 'برابر', 'بڑا',
     'بڑی', 'بڑے', 'بہت', 'بے', 'تجھے', 'تھا', 'تھوڑا', 'تھوڑی', 'تھوڑے', 'تھی', 'تھے', 'جب', 'جس', 'جن',
     'جنا', 'جنہیں', 'جو', 'حالانکہ', 'خبردار', 'خود', 'درمیان', 'دیر', 'دی', 'دیا', 'دیے', 'دو', 'دن',
     'دوسرا', 'دوسری', 'دوسرے', 'دینا', 'رہا', 'رہے', 'رہی', 'ساتھ', 'سکتا', 'سکتی', 'سے', 'سو', 'ضرور',
     'ظاہر', 'عدم', 'کر', 'کرتا', 'کرتی', 'کرتے', 'کیا', 'کئے', 'کرنا', 'کرنے', 'کو', 'کون', 'کیسا',
     'کیسی', 'کیسے', 'کیوں', 'لئے', 'لگا', 'لیکن', 'مارا', 'میں', 'مگر', 'مجھے', 'مزید', 'مل', 'ملی', 'مناسب',
     'نے', 'والا', 'والی', 'والے', 'وقت', 'وغیرہ', 'وہ', 'وہاں', 'وہی', 'ویسا', 'ویسے',
     'ہر', 'ہم', 'ہمیشہ', 'ہمارا', 'ہماری', 'ہمارے', 'ہی', 'یہ', 'یہاں', 'یہی', 'اب', 'ادھر', 'ارد', 'اصل', 
     'اگرچہ', 'امید', 'انہیں', 'بجائے', 'بدستور', 'بلا', 'تم', 'تمہیں', 'تمہارا', 'تمہاری', 'تمہارے', 'تک', 
     'تھک', 'تو', 'توں', 'تین', 'جائے', 'جبکہ', 'جاتا', 'جاتی', 'جاتے', 'جا', 'جیسا', 'جیسے', 'خاطر', 'خلاف', 
     'خیال', 'در', 'دوران', 'ذرا', 'رکھ', 'رکھتا', 'رکھتی', 'رکھتے', 'رکھنا', 'رکھنے', 'صرف', 'عبارت', 'عین', 
     'غالب', 'غیر', 'فقط', 'قابل', 'قریب', 'کرکے', 'کہیں', 'کہ', 'لگ', 'لگتا', 'لگتی', 'لگتے', 'ملتا', 
     'ملتی', 'ملتے', 'نہایت', 'نیز', 'ہو', 'ہوتا', 'ہوتی', 'ہوتے', 'ہوا', 'ہوئی', 'ہوئے', 'ہونگے', 'ہے', 'ہیں', 
     'بالکل', 'تحت', 'تمام', 'جلدی', 'چونکہ', 'حاصل', 'درست', 'زیادہ', 'پہ', 'پہنچ', 'کافی', 'کام', 'کم', 
     'کب', 'کبھی', 'نزدیک', 'پھر', 'جہاں', 'کس', 'کن', 'کچھ', 'کسے', 'کے', 'گو', 'ہزار', 'ہرگز', 'یعنی', 
     'آئیگا', 'اک', 'ان', 'انہاں', 'اندر', 'ایس', 'باہر', 'جد', 'جدوں', 'جہڑا', 'جہڑے', 'سارے', 'ساڈا', 'سی', 
     'کدی', 'کردے', 'کردی', 'کرنا', 'کرن', 'کرو', 'کی', 'کیویں', 'نال', 'ہن', 'ہو', 'ہون', 'ہوں', 'بندہ', 'پر', 'جائے'
     'کرے','وه','گیا ','بتا','ہے','ہا',' نے','اسے','ہے','ہاہاہاہاہاہاہاہا','ہاہاہاہاہاہا','اللہ',
 ]
urdu_stopwords2= [
    "ک", "ی", "ل", "ی", "ے", "ئ", "ترا", "کہتے", "رہا", "تھوڑا", "جائےگی", "یدمز", "ہونا", 
    "یتھوڑ", "ہمیں", "کہتی", "رہاہے", "کسی", "کہتےہیں", "رہب", "توہی", "کرسکے", "یہیں", "ؤکھلا", 
    "رہتے", "توہیں", "کرنے", "ذرا", "یتر", "اتیر", "هالے", "آتے", "ہمی", "کہو", "رہتےہیں", "نہییو", 
    "آتی", "ےتیر", "بعض", "کھولو", "رہنے", "یتیر", "بعدازاں", "آتےہیں", "ترے", "کھولے", "رہے", 
    "هغیر", "آج", "یث", "بعضے", "کہوں", "رہی", "ذکره", "تو", "بڑھ", "یعہزر", "ث", "اسُ", "کہے", 
    "ہواکہ", "اچھب", "یعےزر", "ث", "فلاں", "کہی", "اسے", "کہیں", "زکن", "جاتا", "ہوئی", "اچھے", 
    "الہٰذ", "آخر", "دهیاز", "جارہاہے", "هںتیر", "کوتر", "اسے", "کورا", "سب", "جاؤ", "جلد", "ادھر", 
    "چند", "کورهں", "سکھائے", "جائے", "جلدہی", "اس", "ہمارا", "کوئی", "سکوں", "جائیں", "کرهں", 
    "اکٹھے", "یاکرد", "اکٹھی", "تھاکیا", "سیکھو", "جب", "انہیں", "ل", "چاہتے", "بھر", "اکثر", 
    "چاہتےہیں", "گذشتہ", "آگئے", "یعےکےزر", "انُہیں", "چاہیے", "سابقہ", "آم", "مختلف", "ی", 
    "یسیا", "یافرما", "چلتےبنے", "سکتا", "کرکے", "گرد", "هن", "گئے", "کاہے", "چلو", "جبکہ", 
    "آئے", "م", "چلے", "بارے", "آئی", "ہوچکے", "لگیں", "کرالو", "خوکہ", "گئی", "پ", "ہوچکی", 
    "لو", "کرتے", "دهر", "خو", "پڑتا", "ہورہے", "لوجب", "کرتی", "دهر", "خوامخواہ", "پڑتاہے", 
    "یو", "پڑتے", "ہورہی", "لوجے", "کرتےہو", "یاد", "آککر", "پھر", "هوگئے", "لوسبت", "کرتے", 
    "یتےد", "کہ", "پھرپوچھا", "هوگئی", "لوسہ", "کرتی", "پہلے", "سے", "یتید", "هونے", "لے", 
    "کرنا", "یرد", "ابھی", "پہنچ", "هوئے", "لی", "کرناپڑتیں", "یکهو", "پی", "پہنچوں", "لیتے", 
    "کرنی", "پہنچی", "یکهید", "پڑتی", "هورہی", "کرتاہے", "ہے", "پڑتے", "کرتےہیں", "را", "کئی", 
    "فرد", "چکیں", "سکتے", "کرکے", "موجود", "چکےہے", "مختلف", "پہنچ", "یرد", "کل", "کرتاہے", 
    "کبھی", "ہوچکے", "لگیں", "پڑتا", "ہورہے", "کرتےہیں", "پہنچ", "کبھی", "کرتاہے", "هالے","ہیں","ہے ",
]

combined_stopwords = set(urdu_stopwords + urdu_stopwords1 + urdu_stopwords2)# Combine and deduplicate stop words as set doesn't have duplicates
combined_stopwords_list = list(combined_stopwords)

# Now you can use combined_stopwords for filtering during preprocessing

def remove_stopwords(text, stopwords, sentiment_words):
    if isinstance(text, str):  
        words = text.split()  
        filtered_words = []
        for word in words:
            if word not in stopwords or word in sentiment_words:
                filtered_words.append(word)  
        return ' '.join(filtered_words) 
    return text

sentiment_words = [
    "نہیں", "برا", "خراب", "نفرت", "اچھا", "کامیاب", "خوش", 
    "پیار", "محبت", "تعریف", "عظیم", "خوبصورت", "مایوس"
    # Add more based on your needs
]
df['Cleaned Text'] = df['Text'].apply(lambda x: remove_stopwords(x, combined_stopwords_list, sentiment_words))


# 2) Punctuation, Emojis, and Hashtags:

In [60]:

 # New sentiment dictionary
emoji_sentiment_dict = {
    "positive": [
        "😊", "😁", "😂", "🤣", "😃", "😄", "😅", "😆", "😉", "😍",
        "😘", "🥰", "😻", "😼", "😽", "😇", "🤗", "🤩", "🙌", "👍",
        "👏", "👐", "✌️", "🤝", "🤞", "🤟", "🤘", "✋", "👌", 
        "✨", "🎉", "🎊", "🌈", "🌟", "🌻", "🌼", "🍀", "🍎", "🍉", 
        "🥳", "🥇", "💖", "💞", "💗", "💫", "🦄", "🐶", "🐱", "🐰", 
        "🐼", "🐸", "🍩", "🍕", "🍣", "🍦", "☕", "🌍", "🌞", "🔥", 
        "🥂", "🎈", "🌸", "🌷", "🌹", "🌺", "💐", "💖", "💗", "😹",
        "❤️", "💘", "💞", "💓", "💝", "💟", "🧡", "😎","😜"
        "🤍", "💜", "💚", "💛", "💙", "🩹", "😝", "🙏", "🤲","💯"
    ],
    "negative": [
        "😞", "😠", "😡", "😢", "😭", "😩", "😤", "😳", "😔", "😒", 
        "😣", "😵", "😱", "🥴", "😈", "👿", "👎", "😖", "😫", 
        "😬", "💩", "🌧️", "🌩️", "🌪️", "👺", "⚰️", "☠️", "😾", 
        "😿", "👻", "😵‍💫", "😧","💔"
    ],
    "neutral": [
        "😐", "😑", "😶", "😏", "🤨", "😕", "🙄", "😬", "😮", "😯", 
        "😴", "😪", "🤔", "🤷‍♂️", "🤷‍♀️", "🧐", "😸", "😺", "🦁", 
        "🌍", "🌎", "🌏", "🍔", "🍟", "🌮", "🌯", "🥗", "🌭", 
        "🍜", "🧋", "🚗", "🚕", "🚙", "🚌", "🚎", "🚲", "🏠", 
        "✋", "🖐️", "🖖", "🤚", "👋", "🤙", "👈", "👉", "👆", "👇", 
        "✊", "✌️", "🤞", "🤟", "🖐️", "👌", "👐", "🙌", "👊", "📝",
        "🤜", "🤛", "🦾", "✍️", "💪", "🦵", "🦶", "⚖️", "🔍", 
        "🔒", "🔑", "🔓", "💡", "🔋", "🔌", "📩", "✉️", "🔗", "😓","😲"
    ]
}

def clean_text_with_sentiment(text):
    sentiments = []  
    if pd.isna(text):
        return ''  
   
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE) #remove hyperlinks
    text = re.sub(r"\d+", '', text)  # Remove numerics
    text = re.sub(r"[^\w\s\U0001F600-\U0001F64F\U0001F300-\U0001F5FF]+", '', text) #remove punctuation and symbols
    text = re.sub(r"@\w+", '', text)  # Remove mentions
    text = re.sub(r"#\w+", '', text)  # Remove hashtags   
    text = re.sub(r'\b[A-Za-zA-Z]+\b', '', text)  #Remove English Words
    text = re.sub(r'_', '', text) #Remove underscores
    
    for sentiment, emojis in emoji_sentiment_dict.items():
        for emoji in emojis:
            if emoji in text:
                sentiments.append(sentiment)  # Append the sentiment (positive/negative/neutral) in sentiments list for each document
                text = text.replace(emoji, '')  # Remove the emoji from the text
            
    text = re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF]', '', text)  # Remove unmatched emojis

    text = ' '.join(text.split()) #remove the spaces by splitting and then join the words again with 1 space
    sentiment_str = ' '.join(sentiments) if sentiments else 'NoSentiment' #if the list contains some sentiments,join with single space as a string 
    
    return f"{text.strip()} {sentiment_str}"  # Combine cleaned text with sentiment

df['Cleaned Text'] = df['Cleaned Text'].apply(clean_text_with_sentiment)

# 3) Short Conversations:

In [61]:
def filter_short_posts(text):
    if len(text.split()) < 3:
        return None
    return text

# Apply the filter function to clean the text
df['Cleaned Text'] = df['Cleaned Text'].apply(filter_short_posts)
df.dropna(subset=['Cleaned Text'], inplace=True)

# Create the sentiment column
def assign_sentiment(text):
    if 'NoSentiment' in text:
        return 'neutral'
    elif 'positive' in text:
        return 'positive'
    elif 'negative' in text:
        return 'negative'
    else:
        return 'neutral'  # Default case if none match

df['sentiment'] = df['Cleaned Text'].apply(assign_sentiment) #create new column

# Remove sentiment words from 'Cleaned Text'
df['Cleaned Text'] = df['Cleaned Text'].replace({'positive': '', 'negative': '', 'neutral': '', 'NoSentiment': ''}, regex=True)

# Check sentiment distribution
print("Sentiment distribution after assignment:")

# Save to CSV
df[['Cleaned Text', 'sentiment']].to_csv('C:/Users/admin/OneDrive/Desktop/SEMESTER7/NLP_ass#1/Cleaned Tweets Dataset.csv', index=False, encoding='utf-8-sig')
print(df['sentiment'].value_counts())


Sentiment distribution after assignment:
sentiment
positive    47521
neutral     23280
negative     7204
Name: count, dtype: int64


# 1) Stemming

In [62]:
urdu_stem_dict = {
    # Verbs
    "کرتے": "کر", "کرتی": "کر", "کرتا": "کر", "کرنا": "کر", "کررہا": "کر", "کررہی": "کر", "کررہے": "کر",
    "گیا": "جا", "گئی": "جا", "گئے": "جا", "جانا": "جا", "جاتا": "جا", "جاتی": "جا", "جاتے": "جا", 
    "پڑھتا": "پڑھ", "پڑھتی": "پڑھ", "پڑھتے": "پڑھ", "پڑھنا": "پڑھ", "پڑھا": "پڑھ", "پڑھی": "پڑھ",
    "دیتا": "دے", "دیتی": "دے", "دیتے": "دے", "دینا": "دے", "دے": "دے",
    "لکھتا": "لکھ", "لکھتی": "لکھ", "لکھتے": "لکھ", "لکھنا": "لکھ", "لکھا": "لکھ", "لکھی": "لکھ",
    "دیکھتا": "دیکھ", "دیکھتی": "دیکھ", "دیکھتے": "دیکھ", "دیکھنا": "دیکھ", "دیکھا": "دیکھ", "دیکھی": "دیکھ",
    "چلتا": "چل", "چلتی": "چل", "چلتے": "چل", "چلنا": "چل", "چلا": "چل", "چلی": "چل",
    "بولتا": "بول", "بولتی": "بول", "بولتے": "بول", "بولنا": "بول", "بولا": "بول", "بولی": "بول",
    "سنتا": "سن", "سنتی": "سن", "سنتے": "سن", "سننا": "سن", "سنا": "سن", "سنی": "سن",
    "پہنچتا": "پہنچ", "پہنچتی": "پہنچ", "پہنچتے": "پہنچ", "پہنچنا": "پہنچ", "پہنچا": "پہنچ", "پہنچی": "پہنچ",
    "خریدتا": "خرید", "خریدتی": "خرید", "خریدتے": "خرید", "خریدنا": "خرید", "خریدا": "خرید", "خریدی": "خرید",
    "بیچتا": "بیچ", "بیچتی": "بیچ", "بیچتے": "بیچ", "بیچنا": "بیچ", "بیچا": "بیچ", "بیچی": "بیچ",
    "آتا": "آ", "آتی": "آ", "آتے": "آ", "آنا": "آ", "آیا": "آ", "آئی": "آ",
    "ہوتا": "ہو", "ہوتی": "ہو", "ہوتے": "ہو", "ہونا": "ہو", "ہوا": "ہو", "ہوئی": "ہو",
    "جاتا": "جا", "جاتی": "جا", "جاتے": "جا", "چاہتا": "چاہ", "چاہتی": "چاہ", "چاہتے": "چاہ",
    "کہتا": "کہ", "کہتی": "کہ", "کہتے": "کہ", "کہنا": "کہ", "کہا": "کہ","کہتیں": "کہ",
    "سناتا": "سنا", "سناتی": "سنا", "سناتے": "سنا", "سننا": "سن", "سنایا": "سنا",
    "پکڑتا": "پکڑ", "پکڑتی": "پکڑ", "پکڑتے": "پکڑ", "پکڑنا": "پکڑ", "پکڑا": "پکڑ", "پکڑی": "پکڑ",
    
    # Adjectives
    "اچھا": "اچھا", "اچھی": "اچھا", "اچھے": "اچھا", "خوبصورت": "خوبصورت", "بڑا": "بڑا", "بڑی": "بڑا", "بڑے": "بڑا",
    "چھوٹا": "چھوٹا", "چھوٹی": "چھوٹا", "چھوٹے": "چھوٹا", "لمبا": "لمبا", "لمبی": "لمبا", "لمبے": "لمبا",
    "پیارا": "پیارا", "پیارے": "پیارا", "پیاری": "پیارا", "عظیم": "عظیم", "بری": "برا", "برے": "برا", "برا": "برا",
    
    # Nouns
    "لڑکا": "لڑکا", "لڑکی": "لڑکی", "لڑکے": "لڑکا", "لڑکوں": "لڑکا", "لڑکیوں": "لڑکی", 
    "گھر": "گھر", "کتاب": "کتاب", "کتابیں": "کتاب", "استاد": "استاد", "استادوں": "استاد", "شاگرد": "شاگرد",
    "دوست": "دوست", "دوستوں": "دوست", "پہلی": "پہلا", "پہلے": "پہلا", "شہر": "شہر", "شہروں": "شہر", 
    "آدمی": "آدمی", "آدمیوں": "آدمی", "دوکان": "دوکان", "دوکانیں": "دوکان"
}



In [63]:
def stem_text(text):
    words = text.split()  # Split the text into words
    stemmed_words = []  # Initialize a list for stemmed words
    for word in words:
        # Get the stemmed word or keep the original if not found
        stemmed_words.append(urdu_stem_dict.get(word, word))
    return ' '.join(stemmed_words)  # Join and return the stemmed text

df['stemmed_text'] = df['Cleaned Text'].apply(stem_text)
df[['Cleaned Text', 'sentiment','stemmed_text']].to_csv('C:/Users/admin/OneDrive/Desktop/SEMESTER7/NLP_ass#1/Cleaned Tweets Dataset.csv', index=False, encoding='utf-8-sig')

print(df[ 'stemmed_text'])


0                  جزاک خیر خوشی مقدر دعا قبول فرمائے آمین
1                                                    سچ اے
2                                        بیکری بند بند گھر
3                                              غلط نہیں کہ
4                                              جاہل پٹواری
                               ...                        
83609       بات کا مزہ تب آ آدھی اردو آدھی پنجابی مکس بولے
83610    مریم بھگوڑی کا نہتی لڑکی کیڑا مر جا عزت خاتون ...
83612    فوجہےتوپاکستانہے بلاول زرداری اوقات میرے پاک ف...
83613         انسان سستا محبت نام بکتا مہنگی انسان محبت ہے
83614                               اداسی چیختی قہقہوں درد
Name: stemmed_text, Length: 78005, dtype: object


# 2) Lemmatization

In [64]:
urdu_lemma_dict = {
    # Verbs
    "کرتے": "کرنا", "کرتی": "کرنا", "کرتا": "کرنا", "کررہا": "کرنا", "کررہی": "کرنا", "کررہے": "کرنا",
    "گیا": "جانا", "گئی": "جانا", "گئے": "جانا", "جاتا": "جانا", "جاتی": "جانا", "جاتے": "جانا",
    "پڑھتا": "پڑھنا", "پڑھتی": "پڑھنا", "پڑھتے": "پڑھنا", "پڑھا": "پڑھنا", "پڑھی": "پڑھنا", "پڑھنا": "پڑھنا",
    "دیتا": "دینا", "دیتی": "دینا", "دیتے": "دینا", "دینا": "دینا", "لکھتا": "لکھنا", "لکھتی": "لکھنا", 
    "لکھتے": "لکھنا", "لکھا": "لکھنا", "لکھی": "لکھنا", "لکھنا": "لکھنا",
    "دیکھتا": "دیکھنا", "دیکھتی": "دیکھنا", "دیکھتے": "دیکھنا", "دیکھا": "دیکھنا", "دیکھی": "دیکھنا",
    "چلتا": "چلنا", "چلتی": "چلنا", "چلتے": "چلنا", "چلنا": "چلنا", "بولتا": "بولنا", "بولتی": "بولنا", 
    "بولتے": "بولنا", "بولنا": "بولنا", "سنتا": "سننا", "سنتی": "سننا", "سنتے": "سننا", "سنا": "سننا",
    "پہنچتا": "پہنچنا", "پہنچتی": "پہنچنا", "پہنچتے": "پہنچنا", "پہنچا": "پہنچنا", "پہنچی": "پہنچنا",
    "خریدتا": "خریدنا", "خریدتی": "خریدنا", "خریدتے": "خریدنا", "خریدا": "خریدنا", "خریدی": "خریدنا",
    "بیچتا": "بیچنا", "بیچتی": "بیچنا", "بیچتے": "بیچنا", "بیچا": "بیچنا", "بیچی": "بیچنا",
    "آتا": "آنا", "آتی": "آنا", "آتے": "آنا", "آیا": "آنا", "آئی": "آنا",
    "ہوتا": "ہونا", "ہوتی": "ہونا", "ہوتے": "ہونا", "ہوا": "ہونا", "ہوئی": "ہونا",
    "جاتا": "جانا", "جاتی": "جانا", "جاتے": "جانا", "چاہتا": "چاہنا", "چاہتی": "چاہنا", "چاہتے": "چاہنا",
    "کہتا": "کہنا", "کہتی": "کہنا", "کہتے": "کہنا", "کہا": "کہنا", "کہنا": "کہنا",
    "پکڑتا": "پکڑنا", "پکڑتی": "پکڑنا", "پکڑتے": "پکڑنا", "پکڑا": "پکڑنا", "پکڑی": "پکڑنا",
    "سناتا": "سننا", "سناتی": "سننا", "سناتے": "سننا", "سنایا": "سننا", "سننا": "سننا",
    
    # Adjectives
    "اچھا": "اچھا", "اچھی": "اچھا", "اچھے": "اچھا", "خوبصورت": "خوبصورت", "بڑا": "بڑا", "بڑی": "بڑا", "بڑے": "بڑا",
    "چھوٹا": "چھوٹا", "چھوٹی": "چھوٹا", "چھوٹے": "چھوٹا", "لمبا": "لمبا", "لمبی": "لمبا", "لمبے": "لمبا",
    "پیارا": "پیارا", "پیارے": "پیارا", "پیاری": "پیارا", "عظیم": "عظیم", "برا": "برا", "بری": "برا", "برے": "برا",
    "چھوٹا": "چھوٹا", "لمبا": "لمبا", "کمزور": "کمزور", "طاقتور": "طاقتور", "ذہین": "ذہین", "سست": "سست", "محنتی": "محنتی",
    
    # Nouns
    "لڑکا": "لڑکا", "لڑکی": "لڑکی", "لڑکے": "لڑکا", "لڑکوں": "لڑکا", "لڑکیوں": "لڑکی", 
    "گھر": "گھر", "کتاب": "کتاب", "کتابیں": "کتاب", "استاد": "استاد", "استادوں": "استاد", 
    "شاگرد": "شاگرد", "دوست": "دوست", "دوستوں": "دوست", "پہلی": "پہلا", "پہلے": "پہلا", 
    "شہر": "شہر", "شہروں": "شہر", "آدمی": "آدمی", "آدمیوں": "آدمی", "دوکان": "دوکان", "دوکانیں": "دوکان",
    "لڑکیاں": "لڑکی", "دوستیاں": "دوستی", "اساتذہ": "استاد", "مددگار": "مددگار", "بچہ": "بچہ", 
    "بچے": "بچہ", "بچی": "بچہ", "بچیوں": "بچی", "کتابیں": "کتاب", "کپڑا": "کپڑا", "کپڑے": "کپڑا"
}


In [65]:
def lemmatize_text(text):
    words = text.split()  # Split the text into words
    lemmatized_words = []  # Initialize a list for lemmatized words
    for word in words:
        # Get the lemmatized word or keep the original if not found
        lemmatized_words.append(urdu_lemma_dict.get(word, word))
    return ' '.join(lemmatized_words)  # Join and return the lemmatized text

df['lemmatized_text'] = df['Cleaned Text'].apply(lemmatize_text)

df[['Cleaned Text', 'sentiment','stemmed_text','lemmatized_text']].to_csv('C:/Users/admin/OneDrive/Desktop/SEMESTER7/NLP_ass#1/Cleaned Tweets Dataset.csv', index=False, encoding='utf-8-sig')
print(df[ 'lemmatized_text'])


0                  جزاک خیر خوشی مقدر دعا قبول فرمائے آمین
1                                                    سچ اے
2                                        بیکری بند بند گھر
3                                           غلط نہیں کہتیں
4                                              جاہل پٹواری
                               ...                        
83609     بات کا مزہ تب آنا آدھی اردو آدھی پنجابی مکس بولے
83610    مریم بھگوڑی کا نہتی لڑکی کیڑا مر جانا عزت خاتو...
83612    فوجہےتوپاکستانہے بلاول زرداری اوقات میرے پاک ف...
83613         انسان سستا محبت نام بکتا مہنگی انسان محبت ہے
83614                               اداسی چیختی قہقہوں درد
Name: lemmatized_text, Length: 78005, dtype: object


# 1) Tokenization

In [66]:

# Tokenize the cleaned_text column
df['tokenized_text'] = df['lemmatized_text'].apply(lambda x: word_tokenize(x))

# Display the result
print(df['tokenized_text'].head(10))

0      [جزاک, خیر, خوشی, مقدر, دعا, قبول, فرمائے, آمین]
1                                              [سچ, اے]
2                                [بیکری, بند, بند, گھر]
3                                    [غلط, نہیں, کہتیں]
4                                        [جاہل, پٹواری]
5     [رات, سونے, ریٹویٹ, فالوورز, کا, اضافہ, یقینی,...
6                    [مریم, نواز, طرح, اسکے, حسن, جیلس]
7       [رخصت, آنکھ, ملا, نہیں, جانا, جانا, نہیں, جانا]
9                                   [تانوں, صحیح, بلاک]
10                         [وی, کھول, آدھے, گل, اچ, پا]
Name: tokenized_text, dtype: object


# 2) Tf-IDF (Term Frequency-Inverse Document Frequency):


In [67]:
#words that are both frequent within specific documents and unique across the whole set

vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform(df['lemmatized_text'])  #fit the column and transform it into a TF-IDF sparse matrix

tfidf_scores = tfidf_matrix.sum(axis=0) #sum of each word's value across all documents so it can be represented by 1 value

tfidf_scores_transposed = tfidf_scores.T #turn into a  columns so that it can be stored as dataframe column

feature_names = vectorizer.get_feature_names_out() #get feature name to label each score with the word it represents

tfidf_df = pd.DataFrame(tfidf_scores_transposed, index=feature_names, columns=["TF-IDF Score"])

top_10_df = tfidf_df.sort_values(by="TF-IDF Score", ascending=False).head(10)

print(top_10_df)



       TF-IDF Score
نہیں    2045.992794
کا      1669.614345
ہے      1149.234102
نہ       897.683708
جی       767.512248
جانا     757.707836
گا       720.920963
شکریہ    710.499625
فالو     710.102811
بات      703.444939


# 3) Word2Vec Model

In [68]:
import itertools

# Sample hyperparameters to test
vector_sizes = [50, 100, 200]
windows = [3, 5, 10]
min_counts = [1, 3, 5]

# Iterate over all combinations of parameters
for vector_size, window, min_count in itertools.product(vector_sizes, windows, min_counts):
    print(f"Training Word2Vec with vector_size={vector_size}, window={window}, min_count={min_count}")
    model = Word2Vec(df['tokenized_text'], vector_size=vector_size, window=window, min_count=min_count, workers=4, sg=1)# sg=0 means CBOW; 1 means Skip-gram
    similar_words = model.wv.most_similar('اچھا', topn=5)
    print(similar_words)

Training Word2Vec with vector_size=50, window=3, min_count=1
[('سوری', 0.8570399880409241), ('کتنا', 0.8381215929985046), ('آج', 0.8362111449241638), ('سوچو', 0.8347245454788208), ('دفع', 0.8245216608047485)]
Training Word2Vec with vector_size=50, window=3, min_count=3
[('آج', 0.8205878734588623), ('سوری', 0.8168452382087708), ('سمجھے', 0.7721920609474182), ('کتنا', 0.7716867327690125), ('سوچتا', 0.7678647637367249)]
Training Word2Vec with vector_size=50, window=3, min_count=5
[('سمجھدار', 0.7853494882583618), ('سوری', 0.7838637828826904), ('خاصہ', 0.7731002569198608), ('کتنا', 0.7720376253128052), ('ہاہا', 0.7590405344963074)]
Training Word2Vec with vector_size=50, window=5, min_count=1
[('سوری', 0.8486835360527039), ('سوچتا', 0.8366994261741638), ('کتنا', 0.8316372632980347), ('سمجھا', 0.822475790977478), ('خاصہ', 0.8171761631965637)]
Training Word2Vec with vector_size=50, window=5, min_count=3
[('سوری', 0.836127519607544), ('آج', 0.7995980978012085), ('کتنا', 0.7899698615074158), ('

# PHASE-4

# 1) Unigram, Bigram, and Trigram Analysis

In [70]:
# Initialize lists for unigrams, bigrams, and trigrams
bigrams_list = []
trigrams_list = []
unigrams_list = []

# Extract unigrams, bigrams, and trigrams
for row in df['tokenized_text']:
    # Collect unigrams
    unigrams_list.extend(row)  # Append all items in the row to unigrams_list
    
    # Create bigrams
    bigrams = list(ngrams(row, 2))
    bigrams_list.extend(bigrams)  # Append all bigrams to the bigrams_list
    
    # Create trigrams
    trigrams = list(ngrams(row, 3))
    trigrams_list.extend(trigrams)  # Append all trigrams to the trigrams_list

# Calculate frequency of unigrams, bigrams, and trigrams as key value pairs
unigram_freq = Counter(unigrams_list)
bigram_freq = Counter(bigrams_list)
trigram_freq = Counter(trigrams_list)

# Print the top 10 most common unigrams
print("Top 10 most common unigrams:")
for unigram, freq in unigram_freq.most_common(10):
    print(f"{unigram}: {freq}")

# Print the top 10 most common bigrams
print("\nTop 10 most common bigrams:")
for bigram, freq in bigram_freq.most_common(10):
    print(f"{bigram}: {freq}")

# Print the top 10 most common trigrams
print("\nTop 10 most common trigrams:")
for trigram, freq in trigram_freq.most_common(10):
    print(f"{trigram}: {freq}")


Top 10 most common unigrams:
نہیں: 14983
کا: 14666
ہے: 7038
نہ: 5316
جانا: 4009
گا: 3900
بات: 3299
جی: 3256
فالو: 3172
دل: 3112

Top 10 most common bigrams:
('فالو', 'بیک'): 1397
('صلی', 'علیہ'): 799
('عمران', 'خان'): 781
('فالو', 'فالو'): 766
('نواز', 'شریف'): 743
('سندھ', 'پولیس'): 729
('والوں', 'فالو'): 670
('علیہ', 'وآلہ'): 650
('وآلہ', 'وسلم'): 605
('کا', 'اضافہ'): 587

Top 10 most common trigrams:
('فالو', 'فالو', 'بیک'): 710
('علیہ', 'وآلہ', 'وسلم'): 592
('فالورز', 'کا', 'اضافہ'): 548
('والوں', 'فالو', 'فالو'): 451
('لسٹ', 'شامل', 'ہونے'): 438
('ریٹوئیٹ', 'ریٹوئیٹ', 'والوں'): 438
('ریٹوئیٹ', 'والوں', 'فالو'): 434
('شامل', 'ہونے', 'لیے'): 425
('ڈی', 'مینشن', 'ریٹوئیٹ'): 422
('ہونے', 'لیے', 'ڈی'): 420


# 1) Sentiment Classification Model

In [71]:
# Encode the sentiment labels
le = LabelEncoder()
df['sentiment'] = le.fit_transform(df['sentiment'])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df['tokenized_text'], df['sentiment'], test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer(max_features=9000, ngram_range=(1, 2))
X_train_vectors = vectorizer.fit_transform([' '.join(text) for text in X_train])
X_test_vectors = vectorizer.transform([' '.join(text) for text in X_test])

nb_model = MultinomialNB()
nb_model.fit(X_train_vectors, y_train)

y_train_pred = nb_model.predict(X_train_vectors)
y_test_pred = nb_model.predict(X_test_vectors)

# Evaluate the model on training data
train_accuracy = accuracy_score(y_train, y_train_pred)#correctly predicts
train_precision = precision_score(y_train, y_train_pred, average='macro')#how many of the predicted positives were actually positive.
train_recall = recall_score(y_train, y_train_pred, average='macro') #how many actual positives your model caught.
train_f1 = f1_score(y_train, y_train_pred, average='macro')
print("For Training data")
print("Training Accuracy:", train_accuracy)
print("Training Precision:", train_precision)
print("Training Recall:", train_recall)
print("Training F1-score:", train_f1)

# Evaluate the model on test data
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, average='macro')
test_recall = recall_score(y_test, y_test_pred, average='macro')
test_f1 = f1_score(y_test, y_test_pred, average='macro')

print("For Testing data")
print("Test Accuracy:", test_accuracy)
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test F1-score:", test_f1)

For Training data
Training Accuracy: 0.6752291519774374
Training Precision: 0.6876243927794122
Training Recall: 0.4498606178965136
Training F1-score: 0.4595913163274701
For Testing data
Test Accuracy: 0.647073905518877
Test Precision: 0.6125933726530964
Test Recall: 0.41625204397328774
Test F1-score: 0.41523061318236937
